In [5]:
from pismragis.processing import convert_netcdf_to_dataframe

In [11]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = convert_netcdf_to_dataframe(infiles)
df.to_parquet("../tests/data/test_scalar.parquet")
df.to_csv("../tests/data/test_scalar.csv")



Processing files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 95.11it/s]


Program finished in 0 seconds


In [12]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_1989-1-1_YM.nc")
df = convert_netcdf_to_dataframe(infiles)
df.to_parquet("../tests/data/test_scalar_YM.parquet")
df.to_csv("../tests/data/test_scalar_YM.csv")



Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 408.21it/s]

Program finished in 0 seconds


In [ ]:
infiles = glob("../hindcasts/2023_04_ocean_simple/scalar/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")

In [ ]:
def run_stats(infiles: list, stats_vars: list = ["processor_hours", "wall_clock_hours", "model_years_per_processor_hour"]):
    dfs = []
    for m_file in infiles:
        print(m_file)
        with xr.open_dataset(m_file) as ds:
            dfs.append(pd.DataFrame(data=np.array([ds["run_stats"].attrs[stats_var] for stats_var in stats_vars]).reshape(1, -1), columns=stats_vars))
    return pd.concat(dfs).reset_index(drop=True)

In [ ]:
df = run_stats(infiles)

In [ ]:
import xarray as xr
import pandas as pd

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=len(stats_vars), sharey="row", figsize=[6.2, 2.0])
[sns.histplot(data=df[df[stats_var] > 0][stats_var], ax=axs[i]) for i, stats_var in enumerate(stats_vars)]

In [ ]:
from pismragis.stats import run_stats
import numpy as np

In [ ]:
from glob import glob
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = run_stats(infiles)

In [ ]:
def run_stats(
    infiles: list,
    stats_vars: list = [
        "processor_hours",
        "wall_clock_hours",
        "model_years_per_processor_hour",
    ],
):
    """
    Collect PISM run_stats for a list of files and returns a DataFrame
    """
    dfs = []
    for m_file in infiles:
        with xr.open_dataset(m_file) as ds:
            dfs.append(
                pd.DataFrame(
                    data=np.array(
                        [ds["run_stats"].attrs[stats_var] for stats_var in stats_vars]
                    ).reshape(1, -1),
                    columns=stats_vars,
                )
            )
    return pd.concat(dfs).reset_index(drop=True)


In [ ]:
df.to_csv("../tests/data/test_run_stats.csv", index=False)

In [ ]:
df.to_parquet("../tests/data/test_scalar_YM.parquet")

In [ ]:
def ncfile2dataframe(infile):
    if os.path.isfile(infile):
        with xr.open_dataset(infile) as ds:
            m_id = int(re.search("id_(.+?)_", infile).group(1))
            m_dx = int(re.search("gris_g(.+?)m", infile).group(1))
            datetimeindex = ds.indexes["time"]
            nt = len(datetimeindex)
            id_S = pd.Series(data=np.repeat(m_id, nt), index=datetimeindex, name="id")
            S = [id_S]
            for m_var in ds.data_vars:
                if m_var not in ("time_bounds", "time_bnds", "timestamp", "run_stats", "pism_config"):
                    if hasattr(ds[m_var], "units"):
                        m_units = ds[m_var].units
                        m_S_name = f"{m_var} ({m_units})"
                    else:
                        m_units = ""
                        m_S_name = f"{m_var}"
                    data = np.squeeze(ds[m_var].values)
                    m_S = pd.Series(data=data, index=datetimeindex, name=m_S_name)
                    S.append(m_S)
            df = pd.concat(S, axis=1).reset_index()
            df["resolution_m"] = m_dx
        return pd.concat(S, axis=1).reset_index()


In [ ]:
df.to_csv("../tests/data/test_scalar_YM.csv", index=True)

In [ ]:
    df = pd.read_parquet("../tests/data/test_scalar_YM.parquet")
    ensemble_file = "../tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    sensitivity_df = sensitivity_analysis(
        df, ensemble_file=ensemble_file, n_jobs=4)


In [ ]:
df.to_parquet?

In [ ]:
import re

In [ ]:
re.search("id_(.+?)_", infiles[0]).groups()

In [ ]:
import pandas as pd
from pandas.api.types import is_string_dtype
from SALib.analyze import delta
import numpy as np
import pathlib
from joblib import Parallel, delayed

from tqdm import tqdm

from pismragis.processing import tqdm_joblib
from pismragis.sensitivity import sensitivity_analysis

In [ ]:
sensitivity_df

In [ ]:
sensitivity_df[0].to_parquet("../tests/data/test_sensitivity.parquet")

In [ ]:
    df = pd.read_parquet("../tests/data/test_scalar_YM.parquet")
    df_true = pd.read_parquet("../tests/data/test_sensitivity.parquet")
    ensemble_file = "../tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    sensitivity_df, _ = sensitivity_analysis(
        df, ensemble_file=ensemble_file, n_jobs=4
    )


In [ ]:
sensitivity_df.mean()

In [ ]:
sensitivity_df.columns

In [ ]:
sens_vars = ['vcm', 'gamma_T', 'thickness_calving_threshold', 'ocean_file', 'sia_e',
       'ssa_n', 'pseudo_plastic_q', 'till_effective_fraction_overburden',
       'phi_min', 'phi_max', 'z_min', 'z_max']
sensitivity_df[sens_vars].mean()

In [ ]:
from numpy.testing import 

In [ ]:
np.testing.assert_array_almost_equal?

In [ ]:
-636500 + 1800/2

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset("../hindcasts/2023_04_ocean_simple_20/scalar/ts_gris_g1800m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc")

In [ ]:
month_length = ds.time.dt.days_in_month

In [ ]:
# Calculate the weights by grouping by 'time.season'.
weights = (
    month_length.groupby("time.year") / month_length.groupby("time.year").sum()
)

In [ ]:
weights

In [ ]:
ds_weighted = (ds * weights).groupby("time.year").sum(dim="time")

In [ ]:
ds_weighted

In [ ]:
ds.time.dt.days_in_month

In [ ]:
ds.time.dt.daysinmonth[0:33]

In [ ]:
weights = (
    month_length.groupby("time.month") / month_length.groupby("time.month").sum()
)

In [ ]:
ds_weighted = (ds * weights).groupby("time.month").mean(dim="time")

In [ ]:
ds_weighted

In [ ]:
ds_weighted

In [ ]:
40 * 12

In [ ]:
40 * 365

In [ ]:
ds.resample(time="1YS").mean()["limnsw"]

In [ ]:
def ncfile2dataframe(infile):
    if os.path.isfile(infile):
        with xr.open_dataset(infile) as ds:
            ds = ds.resample(time="1M").mean()
            m_id_re = re.search("id_(.+?)_", infile)
            assert m_id_re is not None
            m_id = int(m_id_re.group(1))
            m_dx_re = re.search("gris_g(.+?)m", infile)
            assert m_dx_re is not None
            m_dx = int(m_dx_re.group(1))
            datetimeindex = ds.indexes["time"]
            nt = len(datetimeindex)
            id_S = pd.Series(data=np.repeat(m_id, nt), index=datetimeindex, name="id")
            S = [id_S]
            for m_var in ds.data_vars:
                if m_var not in (
                    "time_bounds",
                    "time_bnds",
                    "timestamp",
                    "run_stats",
                    "pism_config",
                ):
                    if hasattr(ds[m_var], "units"):
                        m_units = ds[m_var].units
                        m_S_name = f"{m_var} ({m_units})"
                    else:
                        m_units = ""
                        m_S_name = f"{m_var}"
                    data = np.squeeze(ds[m_var].values)
                    m_S = pd.Series(data=data, index=datetimeindex, name=m_S_name)
                    S.append(m_S)
            df = pd.concat(S, axis=1).reset_index()
            df["resolution_m"] = m_dx
        return pd.concat(S, axis=1).reset_index()


In [ ]:
ncfile2dataframe("../hindcasts/2023_04_ocean_simple_20/scalar/ts_gris_g1800m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc")

In [ ]:
import os
import re